In [1]:
import sys
sys.path.append('..')

import pandas as pd
import gc
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from utils.datapreprocessor import SoapiePreprocessor
from utils.datasets import dataset_dict, speech_file_to_array_fn, get_random_elements, down_sample_labels_fn

/home/geoff/anaconda3/envs/penguin/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataframes

In [2]:
engzul_preprocessor = SoapiePreprocessor(lang_pair='cs_engzul')

In [3]:
df_train, df_dev, df_test = engzul_preprocessor.generate_split_dataframes(save_csv=True)

100%|██████████| 768/768 [00:07<00:00, 102.43it/s]


In [7]:
del df_train, df_dev, df_test
gc.collect()

690

### Dataset construction

In [1]:
path = '/media/geoff/datasets/soapies_balanced_corpora/cs_engzul_balanced/lang_targs_mult/cs_engzul_trn.pkl'
df_trn = pd.read_pickle(path)
path = '/media/geoff/datasets/soapies_balanced_corpora/cs_engzul_balanced/lang_targs_mult/cs_engzul_dev.pkl'
df_dev = pd.read_pickle(path)
path = '/media/geoff/datasets/soapies_balanced_corpora/cs_engzul_balanced/lang_targs_mult/cs_engzul_tst.pkl'
df_tst = pd.read_pickle(path)

dataset = dataset_dict(df_trn, df_dev, df_tst)

del df_trn, df_dev, df_tst
gc.collect()

NameError: name 'pd' is not defined

In [3]:
# dataset = dataset.map(speech_file_to_array_fn, num_proc=1)
# dataset = dataset.map(down_sample_labels_fn, num_proc=1)

Parameter 'function'=<function down_sample_labels_fn at 0x7f0532838b80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
8381ex [01:05, 127.47ex/s]
225ex [00:01, 153.48ex/s]
768ex [00:05, 136.37ex/s]


In [4]:
ds_train = dataset['train'].to_pandas()
ds_train.head()

,audio_fpath,tgts,ds_tgts
0,/media/geoff/datasets/soapies_balanced_corpora...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,/media/geoff/datasets/soapies_balanced_corpora...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,/media/geoff/datasets/soapies_balanced_corpora...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
3,/media/geoff/datasets/soapies_balanced_corpora...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
4,/media/geoff/datasets/soapies_balanced_corpora...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."


In [5]:
from torch.nn.utils.rnn import pad_sequence

class CSDataset(Dataset):
    def __init__(self, ds_train, transform=False):
        self.transform = transform
        self.ds_train = ds_train

        #self.audio_path = dataset['audio_fpath']
        #self.speech = dataset['speech']
        #self.target = dataset['ds_tgts']
    
    def __len__(self):
        return len(self.ds_train)
    
    def __getitem__(self, idx):
        audio, sr = torchaudio.load(self.ds_train.audio_fpath.iloc[idx])
        #audio = self.speech[idx][0]
        if self.transform:
            pass
        return audio[0], torch.tensor(self.ds_train.ds_tgts.iloc[idx], dtype=torch.long)

def collator(batch):
    xx = [s[0] for s in batch]
    xx_ll = [len(s[0]) for s in batch]
    yy = [s[1] for s in batch]
    yy_ll = [len(s[1]) for s in batch]

    xx_ll = torch.tensor(xx_ll, dtype=torch.float)
    yy_ll = torch.tensor(yy_ll, dtype=torch.float)

    return pad_sequence(xx, batch_first=True), xx_ll, pad_sequence(yy, batch_first=True, padding_value=1), yy_ll

In [6]:
dataset_trn = CSDataset(ds_train)

In [23]:
# from torch.utils.data.sampler import BatchSampler, RandomSampler
# sampler = BatchSampler(RandomSampler(dataset_trn), batch_size=16, drop_last=False)

In [7]:
train_dataloader = DataLoader(dataset_trn, batch_size=16, collate_fn=collator, num_workers=12)

In [19]:
import time

start = time.perf_counter()

for i, batch in enumerate(train_dataloader):
    x, x_l, y, y_l = batch
    
    # print(x.shape)
    # print(x_l.shape)
    # print(y.shape)
    # print(y_l.shape)
    #break

end = time.perf_counter()
print(start-end)

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
tensor([124., 141., 110.,  96.,  91., 217.,  93., 166.,  89., 117., 108., 102.,
        105.,  82.,  88., 137.])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])
tensor([ 81.,  81., 101.,  80.,  85.,  98.,  90., 102., 100.,  23.,  85.,  84.,
         81.,  90.,  82.,  87.])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [2, 2, 2,  ..., 1, 1, 1],
        [2, 2, 2,  ..., 1, 1, 1],
        [2, 2, 2,  ..., 1, 1, 1]])
tensor([127., 102., 150.,  16.,  31.,  22.,  21.,  14.,   9.,  16.,  11.,  12.,
         12.,  28.,  29.,  30.])
tensor(

In [18]:
x.shape

torch.Size([16, 148400])